In [1]:
from tkinter import *
from tkinter.filedialog import askopenfilename
import math
import os

file=""
#anlyseButton=None
AnaliseLable=None
class FullScreenApp(object):
    def __init__(self, master, **kwargs):
        self.master=master
        master.title("Log Analizer")
        pad=3
        self._geom='200x200+0+0'
        master.geometry("{0}x{1}+0+0".format(
            master.winfo_screenwidth()-pad, master.winfo_screenheight()-pad))
        master.bind('<Escape>',self.toggle_geom)            
    def toggle_geom(self,event):
        geom=self.master.winfo_geometry()
        print(geom,self._geom)
        self.master.geometry(self._geom)
        self._geom=geom
        

In [23]:
class Menubar(object):
    def __init__(self, master):
        self.master=master
        menu = Menu(root)
        root.config(menu=menu)
        subMenu = Menu(menu)
        menu.add_cascade(label="File",menu=subMenu)
        subMenu.add_command(label="Open File",command=self.LoadFile)
        #subMenu.add_command(label="Save Output",command=doNothing)
        #subMenu.add_command(label="Print",command=printFile)
        subMenu.add_separator()
        subMenu.add_command(label="Exit",command=self.master.destroy)
        master.bind('<Escape>', self.close)
        
    def close(self, event=None):
        self.master.destroy()
        
    def LoadFile(self):
        global file
        global root
        global mainWindow
        #root.filename = askopenfilename(initialdir = "/",
        #                                title = "Select file",
        #                                filetypes = (("log files","*.log"),("all files","*.*")))
        #
        root.filename="/home/aniket/eCommunicationsNSMObjMgr_enu_0033.log"
        openFile = open(root.filename, "r")
        file = openFile.readlines()
        loadFile = mainWindow.changefilename(root.filename)
        #addFileToWindow(root.filename,FileLabels)

In [162]:
class MainWindow(object):
    def __init__(self, master):
        global screenWidth
        global screenheight
        global AnaliseLable
        self.FilesFrame = Frame(root)
        self.FileWindow = Text(self.FilesFrame,width = math.floor(screenWidth / 50),
                               height= screenheight, bg="#778a98", fg = "white")
        self.FileWindow.insert(INSERT, "Files \n")
        self.FileWindow.config(state=DISABLED,font="Helvetica 15 bold")
        self.FileWindow.pack(fill=Y)
        #FileWindow.config(state=NORMAL)
        #FileWindow.insert(END, "Aniket is the greatest")
        #FileWindow.config(state=DISABLED)
        self.FilesFrame.pack(side=LEFT,fill=Y)
        
        self.TopFrame = Frame(root,bg = "#CB8B8B")
        self.AnaliseLable = Label(self.TopFrame,text = "No data to Anayse...Please select a file",
                                  font="Times 15 bold",bg = "#CB8B8B")
        self.AnaliseLable.pack(side=LEFT,padx=10,pady=10)
        self.anlyseButton = Button(self.TopFrame,text="Analise",font="Helvetica 15 bold",command=self.Analyze)
        self.anlyseButton.config(state=DISABLED)
        self.anlyseButton.pack(side=LEFT)
        self.TopFrame.pack(side=TOP,fill=X)
        
        self.WrapperFrame = Frame(root,bg="#604f4f")
        self.EvtContextFrame = ScrollableFrame(root,self.WrapperFrame,"EventContext")
        self.EvtContextFrame.forgetIt()
        self.InstanceFrame = ScrollableFrame(root,self.WrapperFrame,"Process Instances")
        self.InstanceFrame.forgetIt()
        self.StepsFrame = ScrollableFrame(root,self.WrapperFrame,"Process Steps")
        self.StepsFrame.forgetIt()
        
        self.WrapperFrame.pack(side=TOP,fill=X)
        
        self.LogsFrame = Frame(root,bg = "#383535")
        self.LogWindow = Text(self.LogsFrame, width=math.floor(screenWidth / 10),
                              height=math.floor(screenheight / 20))
        self.LogWindow.insert(INSERT, "Hello.....")        
        self.LogWindow.insert(END, "You will be able to see logs over here..")
        self.LogWindow.config(state=DISABLED)
        self.LogWindow.pack(side=RIGHT,expand=True)
        self.LogsFrame.pack(fill="both")
        self.LogsFrame.pack_forget()
    
    def changefilename(self,filename):
        self.FileWindow.config(state=NORMAL)
        if(len(self.FileWindow.get('1.0',END)) > 8):
            self.FileWindow.delete('1.0',END)
            self.FileWindow.insert(INSERT, "Files \n")
        self.FileWindow.insert(END, filename)
        self.FileWindow.config(state=DISABLED)
        self.AnaliseLable.config(text='Log file is ready to be analised')
        self.anlyseButton.config(state=NORMAL)
        
    def Analyze(self):
        anaysis = analysis(self)
        anaysis.ListEventContext(self)
        self.EvtContextFrame.showIt()

In [193]:
class ScrollableFrame(object):
    def __init__(self, master,frmeName,FrameDispay):
        global screenWidth
        self.DisplayText=FrameDispay
        self.myframe=Frame(frmeName,relief=GROOVE,bd=1,bg = "Blue")
        self.myframe.pack(side=LEFT,padx=2,pady=5)
        self.canvas=Canvas(self.myframe)
        
        self.frame=Frame(self.canvas)
        myscrollbar=Scrollbar(self.myframe,orient="vertical",command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=myscrollbar.set)
        myscrollbar.pack(side="right",fill="y")
        self.canvas.pack(side="left")
        self.canvas.create_window((0,0),window=self.frame,anchor='nw')
        self.frame.bind("<Configure>",self.myfunction)
        Headerlbl = Label(self.frame,text=FrameDispay,font="Helvetica 15 bold")#.grid(row=0,columnspan=2)
        Headerlbl.pack()
        #self.data()
        self.visible=False
        
    def data(self):
        for i in range(50):
            Label(self.frame,text=str(i)).grid(row=i+1,column=0)
            button = Button(self.frame,text="Hey").grid(row=i+1,column=1)

    def myfunction(self,event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"),
                              width=math.floor(screenWidth / 4.5),height=math.floor(screenWidth / 5))
        
    def forgetIt(self):
        self.myframe.pack_forget()
        self.visible=False
                
    def showIt(self):
        self.myframe.pack(side=LEFT,padx=2,pady=5)
        self.visible=True

In [213]:
class analysis(object):
    def __init__(self,window):
        self.window = window
        
    def ListEventContext(self,window):
        global file
        #eventContextDict = {}
        eventContextNum = 0
        lncnt = 0
        if file is not None:
            for line in file:
                if "EventContext" in line:
                    window.AnaliseLable['text'] = "Analizing Data"
                    eventContextNum += 1
                    #eventContextDict["eventContext"+str(eventContextNum)] = lncnt
                    self.addButton("eventContext "+str(eventContextNum),window.EvtContextFrame.frame,
                                  self.listProcesses,lncnt)
                    self.addLabel("evtCon"+str(eventContextNum),window.EvtContextFrame.frame,line)
                lncnt += 1
                
    def addLabel(self,lName,lWindow,lLine):
        lineContent = lLine.split("\t")
        tail=""
        if 'Instantiating process definition' in lineContent[5]:
            labeltext = lineContent[5][33:68]
            if len(lineContent[5][33:-3]) > 35:
                tail ="..."
        elif 'Instantiating step definition' in lineContent[5]:
            labeltext = lineContent[5][30:65]
            if len(lineContent[5][30:-3]) > 35:
                tail ="..."
        else:
            labeltext = lineContent[5][:35]
            if len(lineContent[5]) > 35:
                tail ="..."
        lName = Label(lWindow,text=labeltext+tail)
        lName.pack()
        
    def addButton(self,bName,bWindow,bCommand,lnNum):
        bName = Button(bWindow,text=bName,command=lambda: bCommand(lnNum))
        bName.pack()
        
    def listProcesses(self,lnNum):
        global file
        end=lnNum
        if self.window.InstanceFrame.visible == True :
            for widget in self.window.InstanceFrame.frame.winfo_children():
                widget.destroy()
            Headerlbl = Label(self.window.InstanceFrame.frame,text=self.window.InstanceFrame.DisplayText,
                              font="Helvetica 15 bold")
            Headerlbl.pack()
        else:
            self.window.InstanceFrame.showIt()
        for line in file[lnNum+1:]:
            if "EventContext" in line:
                break
            end+=1
        
        self.InstantiatingRec(lnNum,end)
            
    def InstantiatingRec(self,start,end):
        global file
        processList={}
        processStarted = False
        lineNum = start
        counter=0
        for line in file[start:end]:
            if processStarted is False:
                if "Instantiating process definition" in line:
                    lineContent = line.split("\t")
                    pname = lineContent[len(lineContent)-1][34:-3]
                    processList[pname]=lineNum
                    self.addButton("Process "+str(counter),self.window.InstanceFrame.frame,self.listSteps,lineNum)
                    self.addLabel(pname,self.window.InstanceFrame.frame,line)
                    counter+=1
                    processStarted = True
            elif "Stopping process instance of '" in line:
                lineContent = line.split("\t")
                lncont = lineContent[len(lineContent)-1]
                x = re.search(r"^Stopping.*"+pname+".*", lncont)#Completed*
                if (x):
                    processStarted = False
            lineNum+=1
    
    def doNothing(self,dummy):
        print("Not doing anything !!")
        
    def listSteps(self,lnNum):
        global file
        end=lnNum
        if self.window.StepsFrame.visible == True :
            for widget in self.window.StepsFrame.frame.winfo_children():
                widget.destroy()
            Headerlbl = Label(self.window.StepsFrame.frame,text=self.window.StepsFrame.DisplayText,
                              font="Helvetica 15 bold")
            Headerlbl.pack()
        else:
            self.window.StepsFrame.showIt()
        for line in file[lnNum+1:]:
            if "EventContext" in line:
                break
            end+=1
        
        self.StepRec(lnNum,end)
        
    def StepRec(self,start,end):
        global file
        stepList={}
        stepStarted = False
        lineNum = start
        counter=0
        for line in file[start:end]:
            if stepStarted is False:
                if "Instantiating step definition" in line:
                    lineContent = line.split("\t")
                    sname = lineContent[len(lineContent)-1][34:-3]
                    stepList[sname]=lineNum
                    self.addButton("Step "+str(counter),self.window.StepsFrame.frame,self.doNothing,lineNum)
                    self.addLabel(sname,self.window.StepsFrame.frame,line)
                    counter+=1
                    stepStarted = True
            elif "Stopping step instance of '" in line:
                lineContent = line.split("\t")
                lncont = lineContent[len(lineContent)-1]
                x = re.search(r"^Stopping.*"+sname+".*", lncont)#Completed*
                if (x):
                    stepStarted = False
            lineNum+=1

In [214]:
from tkinter import ttk
root=Tk()

root.bind('<Escape>', root.destroy)
app=FullScreenApp(root)
screenWidth = root.winfo_screenwidth()
screenheight = root.winfo_screenheight()
menuBar=Menubar(root)
mainWindow=MainWindow(root)
menuBar.LoadFile()
#loadFileName=loadFileName(root,"Aniket")
root.mainloop()

Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
Not doing anything !!
